This code entails data filtering and cleaning that was used to create data for the [Trasnmetrika](https://transmetrika.com/) Belgrade public transit data challenge. 

In [1]:
import pandas as pd
import partridge as ptg
import geopandas
import numpy as np
from datetime import datetime

In [3]:
colnames = ["id", 'stop_id', 'route_path_id', 
            'forecast_time', 'byTelemetry', 'tmId', 'routePathId', 'request_time']

In [5]:
tab = pd.read_csv("forecast.csv", names = colnames, header=None)

In [27]:
tab = tab.reset_index(drop=True)

In [28]:
tab

,id,stop_id,route_path_id,forecast_time,byTelemetry,tmId,routePathId,request_time
0,1,0733a07f-cd14-4fc9-81fe-746951f62fdf,6e223245-fb0c-4a3e-b843-64d894c07592,1644633597,0,0,7e8c4dab-5015-4f71-aed0-09e6f0012d73,1644620502
1,2,0733a07f-cd14-4fc9-81fe-746951f62fdf,79be2042-7c61-4208-89ce-0ef12f417fbd,1644642657,0,0,2d0ac2d3-bb64-46bb-8dd6-196d9195cb47,1644620502
2,3,0004b6de-c70f-4ae7-adf6-0aa4f563d782,74e301c2-e932-4965-b72d-4d03697dd405,1644633537,0,0,59dc26dd-2dfe-4e3d-8f5f-d7c1a9e8f200,1644620502
3,4,0004b6de-c70f-4ae7-adf6-0aa4f563d782,4839019b-d276-4b8a-a1b1-860d70703f93,1644633717,0,0,91529f82-1311-4605-9161-155434d0140b,1644620502
4,5,0004b6de-c70f-4ae7-adf6-0aa4f563d782,2d178ae9-e4c2-4562-b20a-a3b46d4f5cd8,1644635697,0,0,f4ad4d26-e458-4549-b61b-928473300698,1644620502
...,...,...,...,...,...,...,...,...
3479532,3486344,f2ea70b3-6545-4aab-b8fd-788462c2e9a8,bceeb5a3-d4d4-4ad7-b3df-6837942c7c2d,1645115681,0,0,c02f42b4-dd1f-4b7e-8bd7-5d17e15b6a39,1645111705
3479533,3486345,93562e3f-873f-4d09-8264-b735d2ff83a4,dfc7849d-96c0-4345-af14-3ac6a2ff6b84,1645115561,0,0,84e59fdd-d31f-4ffe-b04e-71791aae463e,1645111705
3479534,3486346,93562e3f-873f-4d09-8264-b735d2ff83a4,aa12db25-c5fd-4a26-8d06-6c185f8ba771,1645115621,0,0,9a4140d9-8e67-4d86-b852-4fa95183a4cf,1645111705
3479535,3486347,93562e3f-873f-4d09-8264-b735d2ff83a4,ceb2caaa-75d9-4b6b-bd01-5cbc9a4572e3,1645129601,0,0,748671c9-256e-42bd-be54-e25e76dfd76c,1645111705


In [33]:
import numpy as np
import pandas as pd
from time import time, asctime


def get_cleaned(read_from=None, write_to=None):
    
    df = pd.read_csv(read_from,sep=';', header=0)
    

    start = time()
    print(f'\nCleaner started at: {asctime()}')

    df = df[df['byTelemetry'] == 1]
    data_cleaned = np.array(df.columns)
    i = 0

    for stop in df['stop_id'].unique():
        for route in df[df['stop_id'] == stop]['routePathId'].unique():
            for bus in df[(df['stop_id'] == stop) &
                          (df['routePathId'] == route)]['tmId'].unique():
                dups = df[
                    (df['stop_id'] == stop) &
                    (df['routePathId'] == route) &
                    (df['tmId'] == bus)].sort_values('request_time', ascending=False)

                data_cleaned = np.vstack((data_cleaned, dups.to_numpy()[0, :]))

                for idx, dup in dups.iloc[1:, :].iterrows():
                    if abs(dup['forecast_time'] - float(data_cleaned[-1, 3])) > 1200:
                        data_cleaned = np.vstack((data_cleaned, dup.to_numpy()))
                    i += 1
                    if i % 1000 == 0:
                        elapsed = (time() - start) / 60
                        print(f'{i:,} items handled, {elapsed:.2f} min elapsed,')

    df_cleaned = pd.DataFrame(data_cleaned[1:], columns=data_cleaned[0])
    df_cleaned.drop_duplicates(inplace=True)

    elapsed = (time() - start) / 60
    print(f'Cleaner finished at: {asctime()}\n{elapsed:.2f} min elapsed.')

    df.to_csv(write_to, index=False)


In [34]:
get_cleaned(read_from = "forecast_new_new.csv",
            write_to = "forecast_final.csv")


Cleaner started at: Sun Dec  4 20:35:35 2022
1,000 items handled, 0.18 min elapsed,
2,000 items handled, 0.27 min elapsed,
3,000 items handled, 0.39 min elapsed,
4,000 items handled, 0.52 min elapsed,
5,000 items handled, 0.61 min elapsed,
6,000 items handled, 0.74 min elapsed,
7,000 items handled, 0.85 min elapsed,
8,000 items handled, 0.98 min elapsed,
9,000 items handled, 1.09 min elapsed,
10,000 items handled, 1.26 min elapsed,
11,000 items handled, 1.42 min elapsed,
12,000 items handled, 1.53 min elapsed,
13,000 items handled, 1.71 min elapsed,
14,000 items handled, 1.84 min elapsed,
15,000 items handled, 1.97 min elapsed,
16,000 items handled, 2.08 min elapsed,
17,000 items handled, 2.18 min elapsed,
18,000 items handled, 2.29 min elapsed,
19,000 items handled, 2.38 min elapsed,
20,000 items handled, 2.59 min elapsed,
21,000 items handled, 2.73 min elapsed,
22,000 items handled, 2.83 min elapsed,
23,000 items handled, 2.96 min elapsed,
24,000 items handled, 3.04 min elapsed,
25,

200,000 items handled, 23.25 min elapsed,
201,000 items handled, 23.32 min elapsed,
202,000 items handled, 23.42 min elapsed,
203,000 items handled, 23.53 min elapsed,
204,000 items handled, 23.63 min elapsed,
205,000 items handled, 23.72 min elapsed,
206,000 items handled, 23.82 min elapsed,
207,000 items handled, 23.92 min elapsed,
208,000 items handled, 24.12 min elapsed,
209,000 items handled, 24.22 min elapsed,
210,000 items handled, 24.31 min elapsed,
211,000 items handled, 24.41 min elapsed,
212,000 items handled, 24.50 min elapsed,
213,000 items handled, 24.57 min elapsed,
214,000 items handled, 24.67 min elapsed,
215,000 items handled, 24.75 min elapsed,
216,000 items handled, 24.90 min elapsed,
217,000 items handled, 24.99 min elapsed,
218,000 items handled, 25.06 min elapsed,
219,000 items handled, 25.11 min elapsed,
220,000 items handled, 25.26 min elapsed,
221,000 items handled, 25.41 min elapsed,
222,000 items handled, 25.48 min elapsed,
223,000 items handled, 25.59 min e

396,000 items handled, 43.67 min elapsed,
397,000 items handled, 43.80 min elapsed,
398,000 items handled, 43.91 min elapsed,
399,000 items handled, 44.03 min elapsed,
400,000 items handled, 44.16 min elapsed,
401,000 items handled, 44.27 min elapsed,
402,000 items handled, 44.37 min elapsed,
403,000 items handled, 44.47 min elapsed,
404,000 items handled, 44.55 min elapsed,
405,000 items handled, 44.62 min elapsed,
406,000 items handled, 44.71 min elapsed,
407,000 items handled, 44.81 min elapsed,
408,000 items handled, 44.95 min elapsed,
409,000 items handled, 45.06 min elapsed,
410,000 items handled, 45.21 min elapsed,
411,000 items handled, 45.34 min elapsed,
412,000 items handled, 45.46 min elapsed,
413,000 items handled, 45.57 min elapsed,
414,000 items handled, 45.67 min elapsed,
415,000 items handled, 45.77 min elapsed,
416,000 items handled, 45.86 min elapsed,
417,000 items handled, 45.97 min elapsed,
418,000 items handled, 46.07 min elapsed,
419,000 items handled, 46.17 min e

In [35]:
pd.read_csv("forecast_final.csv")

,id,stop_id,route_path_id,forecast_time,byTelemetry,tmId,routePathId,request_time
0,8,a67e06ee-4fe7-4ee6-bc01-dc428924b4a1,05367e1b-150e-44ab-8c88-ef0cc3d363a7,1644620883,1,1445173,d819c2bc-43a4-4f02-b3e9-28a6f291e9d4,1644620502
1,9,a67e06ee-4fe7-4ee6-bc01-dc428924b4a1,05367e1b-150e-44ab-8c88-ef0cc3d363a7,1644622528,1,1445146,d819c2bc-43a4-4f02-b3e9-28a6f291e9d4,1644620502
2,147,f76c2410-374c-4f0b-b3e4-45523665957f,1401248b-5212-4b51-8c7e-b651dc59bd5f,1644621156,1,1593476,68406a24-07ce-4022-8bea-4911e6f8d8d8,1644620940
3,213,8431e36b-9716-4794-b349-13e69c994324,1401248b-5212-4b51-8c7e-b651dc59bd5f,1644622027,1,1593476,68406a24-07ce-4022-8bea-4911e6f8d8d8,1644620940
4,217,c3b241da-0c88-46d4-a242-2c1a51a30f8c,1401248b-5212-4b51-8c7e-b651dc59bd5f,1644622094,1,1593476,68406a24-07ce-4022-8bea-4911e6f8d8d8,1644620940
...,...,...,...,...,...,...,...,...
493497,999986,be06527b-df8f-489c-b2a3-323c8de88bac,d5a4ba23-7a6a-4fec-af0d-eabeee541eb8,1644668624,1,38448,880dbcbc-5c8b-451b-a07a-3e5378ba6cb4,1644665572
493498,999987,be06527b-df8f-489c-b2a3-323c8de88bac,0ed01412-18d3-4fdc-b3ca-9125b19916c9,1644666654,1,38728,839cc56b-1e24-48c2-ac1c-3ae3b655629c,1644665572
493499,999993,0263b458-56b0-445a-80b6-6b2291ff4255,fa38fbf9-18f6-4be0-92ac-bc1e49d93951,1644665639,1,1304142,3da63d7f-8289-4dc1-ad23-cb8da9b6c8b6,1644665572
493500,999996,0263b458-56b0-445a-80b6-6b2291ff4255,d246bbfd-46dc-4624-9a40-0b83840c7e28,1644665886,1,1151872,05f24588-7c54-4504-9ec6-2afeb5c288f6,1644665572
